Xavier Initialization (Glorot Initialization)
- 임의의 출발점에서 global minima를 찾는 것이 목적
- 문제점: 출발점에 따라 local minima에 빠지거나 saddle point에 도달하는 경우
- 출발점을 잘 고르는데 도움을 준다
- 작동 원리
    - 평균 = 0
    - 분산 = 2 / (Channel_in + Channel_out)
        - Channel_in: input으로 들어가는 channel 갯수
        - Channel_out: output으로 나오는 channel 갯수

He Initialization
- Relu 함수에 특화
- 작동 원리
    - 평균 = 0
    - 분산 = 4 / (Channel_in + Channel_out)

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os


tf.enable_eager_execution()

In [2]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [3]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [4]:
def create_model_function_xavier(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [5]:
def create_model_function_he(label_dim) :
    weight_init = tf.keras.initializers.he_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [6]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [7]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

""" iterator """
train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [9]:
""" Model """
# network = create_model_function_he(label_dim)
network = create_model_function_xavier(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_he'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.4375, train_loss: 2.11379337, train_accuracy: 0.3438, test_Accuracy: 0.1989
Epoch: [ 0] [    1/  468] time: 1.8043, train_loss: 2.02682447, train_accuracy: 0.4766, test_Accuracy: 0.4249
Epoch: [ 0] [    2/  468] time: 2.1631, train_loss: 1.85756433, train_accuracy: 0.6094, test_Accuracy: 0.5519
Epoch: [ 0] [    3/  468] time: 2.5267, train_loss: 1.78941703, train_accuracy: 0.6641, test_Accuracy: 0.5933
Epoch: [ 0] [    4/  468] time: 2.8951, train_loss: 1.74651933, train_accuracy: 0.5703, test_Accuracy: 0.6446
Epoch: [ 0] [    5/  468] time: 3.2562, train_loss: 1.54132771, train_accuracy: 0.7031, test_Accuracy: 0.6968
Epoch: [ 0] [    6/  46

Epoch: [ 0] [   71/  468] time: 25.1341, train_loss: 0.24420710, train_accuracy: 0.9219, test_Accuracy: 0.9133
Epoch: [ 0] [   72/  468] time: 25.4695, train_loss: 0.37932372, train_accuracy: 0.9219, test_Accuracy: 0.9167
Epoch: [ 0] [   73/  468] time: 25.7742, train_loss: 0.21116692, train_accuracy: 0.9375, test_Accuracy: 0.9173
Epoch: [ 0] [   74/  468] time: 26.1412, train_loss: 0.29082599, train_accuracy: 0.9141, test_Accuracy: 0.9189
Epoch: [ 0] [   75/  468] time: 26.4890, train_loss: 0.29240558, train_accuracy: 0.9375, test_Accuracy: 0.9174
Epoch: [ 0] [   76/  468] time: 26.8168, train_loss: 0.31586283, train_accuracy: 0.9297, test_Accuracy: 0.9176
Epoch: [ 0] [   77/  468] time: 27.1465, train_loss: 0.26976991, train_accuracy: 0.9141, test_Accuracy: 0.9155
Epoch: [ 0] [   78/  468] time: 27.4446, train_loss: 0.24579407, train_accuracy: 0.9297, test_Accuracy: 0.9150
Epoch: [ 0] [   79/  468] time: 27.7517, train_loss: 0.31446242, train_accuracy: 0.8984, test_Accuracy: 0.9190
E

Epoch: [ 0] [  145/  468] time: 52.6431, train_loss: 0.24712604, train_accuracy: 0.9453, test_Accuracy: 0.9266
Epoch: [ 0] [  146/  468] time: 53.0101, train_loss: 0.29102883, train_accuracy: 0.9141, test_Accuracy: 0.9273
Epoch: [ 0] [  147/  468] time: 53.3651, train_loss: 0.26488864, train_accuracy: 0.9219, test_Accuracy: 0.9287
Epoch: [ 0] [  148/  468] time: 53.9622, train_loss: 0.29007155, train_accuracy: 0.9219, test_Accuracy: 0.9339
Epoch: [ 0] [  149/  468] time: 54.4388, train_loss: 0.15350929, train_accuracy: 0.9609, test_Accuracy: 0.9372
Epoch: [ 0] [  150/  468] time: 56.3635, train_loss: 0.16443281, train_accuracy: 0.9453, test_Accuracy: 0.9349
Epoch: [ 0] [  151/  468] time: 56.9034, train_loss: 0.18174104, train_accuracy: 0.9531, test_Accuracy: 0.9349
Epoch: [ 0] [  152/  468] time: 57.3215, train_loss: 0.30568454, train_accuracy: 0.9141, test_Accuracy: 0.9350
Epoch: [ 0] [  153/  468] time: 57.7108, train_loss: 0.15419021, train_accuracy: 0.9688, test_Accuracy: 0.9331
E

Epoch: [ 0] [  219/  468] time: 81.3445, train_loss: 0.21305084, train_accuracy: 0.9141, test_Accuracy: 0.9443
Epoch: [ 0] [  220/  468] time: 81.7566, train_loss: 0.14364889, train_accuracy: 0.9766, test_Accuracy: 0.9453
Epoch: [ 0] [  221/  468] time: 82.2019, train_loss: 0.14683741, train_accuracy: 0.9688, test_Accuracy: 0.9448
Epoch: [ 0] [  222/  468] time: 82.5661, train_loss: 0.14940713, train_accuracy: 0.9688, test_Accuracy: 0.9433
Epoch: [ 0] [  223/  468] time: 83.0222, train_loss: 0.18267673, train_accuracy: 0.9453, test_Accuracy: 0.9411
Epoch: [ 0] [  224/  468] time: 83.4268, train_loss: 0.14349571, train_accuracy: 0.9531, test_Accuracy: 0.9383
Epoch: [ 0] [  225/  468] time: 83.8189, train_loss: 0.14321890, train_accuracy: 0.9453, test_Accuracy: 0.9390
Epoch: [ 0] [  226/  468] time: 84.2456, train_loss: 0.22523631, train_accuracy: 0.9375, test_Accuracy: 0.9423
Epoch: [ 0] [  227/  468] time: 84.6696, train_loss: 0.17863247, train_accuracy: 0.9453, test_Accuracy: 0.9432
E

Epoch: [ 0] [  293/  468] time: 111.1869, train_loss: 0.14290336, train_accuracy: 0.9531, test_Accuracy: 0.9506
Epoch: [ 0] [  294/  468] time: 111.6186, train_loss: 0.12482174, train_accuracy: 0.9688, test_Accuracy: 0.9508
Epoch: [ 0] [  295/  468] time: 111.9892, train_loss: 0.23679298, train_accuracy: 0.9375, test_Accuracy: 0.9503
Epoch: [ 0] [  296/  468] time: 112.3543, train_loss: 0.15615027, train_accuracy: 0.9688, test_Accuracy: 0.9485
Epoch: [ 0] [  297/  468] time: 112.6950, train_loss: 0.12261138, train_accuracy: 0.9609, test_Accuracy: 0.9485
Epoch: [ 0] [  298/  468] time: 113.0911, train_loss: 0.20578077, train_accuracy: 0.9375, test_Accuracy: 0.9485
Epoch: [ 0] [  299/  468] time: 113.4563, train_loss: 0.13373350, train_accuracy: 0.9609, test_Accuracy: 0.9504
Epoch: [ 0] [  300/  468] time: 113.8577, train_loss: 0.12761483, train_accuracy: 0.9766, test_Accuracy: 0.9523
Epoch: [ 0] [  301/  468] time: 114.2320, train_loss: 0.09637810, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  367/  468] time: 138.1088, train_loss: 0.09651233, train_accuracy: 0.9766, test_Accuracy: 0.9581
Epoch: [ 0] [  368/  468] time: 138.4347, train_loss: 0.13382463, train_accuracy: 0.9531, test_Accuracy: 0.9578
Epoch: [ 0] [  369/  468] time: 138.7370, train_loss: 0.09593834, train_accuracy: 0.9688, test_Accuracy: 0.9575
Epoch: [ 0] [  370/  468] time: 139.0701, train_loss: 0.08880716, train_accuracy: 0.9688, test_Accuracy: 0.9569
Epoch: [ 0] [  371/  468] time: 139.4092, train_loss: 0.12640718, train_accuracy: 0.9453, test_Accuracy: 0.9586
Epoch: [ 0] [  372/  468] time: 139.7389, train_loss: 0.15768750, train_accuracy: 0.9531, test_Accuracy: 0.9582
Epoch: [ 0] [  373/  468] time: 140.0626, train_loss: 0.14073476, train_accuracy: 0.9688, test_Accuracy: 0.9582
Epoch: [ 0] [  374/  468] time: 140.4384, train_loss: 0.08469723, train_accuracy: 0.9688, test_Accuracy: 0.9587
Epoch: [ 0] [  375/  468] time: 140.7859, train_loss: 0.06321280, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  441/  468] time: 162.0937, train_loss: 0.17342399, train_accuracy: 0.9609, test_Accuracy: 0.9593
Epoch: [ 0] [  442/  468] time: 162.4410, train_loss: 0.14089704, train_accuracy: 0.9375, test_Accuracy: 0.9601
Epoch: [ 0] [  443/  468] time: 162.8279, train_loss: 0.17997187, train_accuracy: 0.9531, test_Accuracy: 0.9611
Epoch: [ 0] [  444/  468] time: 163.2054, train_loss: 0.07196327, train_accuracy: 0.9844, test_Accuracy: 0.9624
Epoch: [ 0] [  445/  468] time: 163.5292, train_loss: 0.10262416, train_accuracy: 0.9609, test_Accuracy: 0.9639
Epoch: [ 0] [  446/  468] time: 163.8677, train_loss: 0.06490514, train_accuracy: 0.9844, test_Accuracy: 0.9641
Epoch: [ 0] [  447/  468] time: 164.2184, train_loss: 0.14873734, train_accuracy: 0.9688, test_Accuracy: 0.9632
Epoch: [ 0] [  448/  468] time: 164.5728, train_loss: 0.05892865, train_accuracy: 0.9922, test_Accuracy: 0.9621
Epoch: [ 0] [  449/  468] time: 164.9330, train_loss: 0.21162492, train_accuracy: 0.9375, test_Accuracy:

정확도 비교
- Random값으로 초기화: 85.35%
- Xavier로 초기화: 96.50%
- He로 초기화: 96.34%